# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.largecore.gth")
# Helium at the center of the box
atoms     = [ElementPsp(:He, pseudopotentials)]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model  = model_DFT(lattice, atoms, positions;
                   functionals=LDA(), symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis; tol)
μref   = dipole(basis, scfres.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770271151884                   -0.53    8.0    162ms
  2   -2.771682358372       -2.85       -1.29    1.0    101ms
  3   -2.771714035301       -4.50       -2.74    1.0   93.5ms
  4   -2.771714711906       -6.17       -3.83    2.0    151ms
  5   -2.771714714917       -8.52       -4.25    2.0    429ms
  6   -2.771714715241       -9.49       -5.04    1.0    120ms
  7   -2.771714715249      -11.10       -6.51    1.0    132ms
  8   -2.771714715250      -12.58       -6.44    2.0    118ms
  9   -2.771714715250      -13.83       -6.99    1.0    100ms
 10   -2.771714715250      -14.88       -7.56    1.0    101ms
 11   -2.771714715250   +  -14.03       -8.54    2.0    117ms


-0.00013457387967525983

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770497442853                   -0.52    9.0    168ms
  2   -2.771778542850       -2.89       -1.33    1.0   98.4ms
  3   -2.771801490165       -4.64       -2.41    1.0   94.3ms
  4   -2.771801997589       -6.29       -3.13    1.0   96.9ms
  5   -2.771802074155       -7.12       -4.49    2.0    116ms
  6   -2.771802074440       -9.54       -4.93    1.0    181ms
  7   -2.771802074473      -10.48       -5.46    2.0    445ms
  8   -2.771802074476      -11.58       -6.30    1.0    140ms
  9   -2.771802074476      -13.01       -7.38    2.0    155ms
 10   -2.771802074476      -14.31       -7.39    1.0    102ms
 11   -2.771802074476   +  -14.40       -8.41    1.0    104ms


0.017612219974150536

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457387967525983
Displaced dipole:  0.017612219974150536
Polarizability :   1.7746793853825795


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response

Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field).

In [5]:
# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

54×54×54×1 Array{Float64, 4}:
[:, :, 1, 1] =
  5.0       5.0       5.0       5.0      …   5.0       5.0       5.0
  4.81481   4.81481   4.81481   4.81481      4.81481   4.81481   4.81481
  4.62963   4.62963   4.62963   4.62963      4.62963   4.62963   4.62963
  4.44444   4.44444   4.44444   4.44444      4.44444   4.44444   4.44444
  4.25926   4.25926   4.25926   4.25926      4.25926   4.25926   4.25926
  4.07407   4.07407   4.07407   4.07407  …   4.07407   4.07407   4.07407
  3.88889   3.88889   3.88889   3.88889      3.88889   3.88889   3.88889
  3.7037    3.7037    3.7037    3.7037       3.7037    3.7037    3.7037
  3.51852   3.51852   3.51852   3.51852      3.51852   3.51852   3.51852
  3.33333   3.33333   3.33333   3.33333      3.33333   3.33333   3.33333
  ⋮                                      ⋱                      
 -3.33333  -3.33333  -3.33333  -3.33333  …  -3.33333  -3.33333  -3.33333
 -3.51852  -3.51852  -3.51852  -3.51852     -3.51852  -3.51852  -3.51852
 -3.7037   -3.7037 

Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

First we do this using the `DFTK.solve_ΩplusK_split`
function provided by DFTK,
which uses an adaptive Krylov subspace algorithm [^HS2025]:

[^HS2025]:
    M. F. Herbst and B. Sun.
    *Efficient Krylov methods for linear response in plane-wave electronic structure calculations.*
    [arXiv 2505.02319](http://arxiv.org/abs/2505.02319)

In [6]:
# Multiply δVext times the Bloch waves, then solve the Dyson equation:
δVψ = DFTK.multiply_ψ_by_blochwave(scfres.basis, scfres.ψ, δVext)
res = DFTK.solve_ΩplusK_split(scfres, -δVψ; verbose=true)

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0   5.77s  Non-interacting
   1        0       1       -0.60     10.0   13.0s  
   2        0       2       -2.42      8.0   567ms  
   3        0       3       -3.54      5.0   101ms  
   4        0       4       -5.33      4.0  97.0ms  
   5        0       5       -7.52      1.0  82.5ms  
   6        0       6      -10.03      1.0  84.7ms  
   7        1       1       -7.37     13.0   311ms  Restart
   8        1       2       -8.94      1.0  93.9ms  
                                      13.0   745ms  Final orbitals


(δψ = Matrix{ComplexF64}[[-0.0022810625090525834 + 0.0023790069248540396im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.22151934654305838 + 0.2159985531530596im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; … ; 0.029526394490517527 + 0.02854012737426297im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; -0.05863607838649198 - 0.05579249261525004im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im]], δρ = [-3.620693992448766e-7 -2.502014505567654e-7 … -4.880643785301705e-8 -2.502015095906026e-7; -6.293616092283937e-7 -4.793035703839599e-7 … -1.2077843199822216e-7 -4.793036141971077e-7; … ; 1.3626259991786124e-7 1.135622952961788e-7 … 4.711989721741241e-8 1.1356228606392175e-7; 1.3305475407079878e-7 1.363264191848558e-7 … 5.200684346183804e-8 1.3632637660066821e-7;;; -2.502014757805875e-7 -1.7404903048183925e-7 … -3.556494726975597e-8 -1.7404907346578694e-7; -4.793035907913793e-7 -3.679731435766836e-7 … -1.1202894067886646e-7 -3.6797317580940876e-7; … ; 1.135622927267842e-7 1.1211098188884472e-7 … 1.0980682404203366e-7 1.121109

From the result of `solve_ΩplusK_split` we can easily compute the polarisabilities:

In [7]:
println("Non-interacting polarizability: $(dipole(basis, res.δρ0))")
println("Interacting polarizability:     $(dipole(basis, res.δρ))")

Non-interacting polarizability: 1.925712536767234
Interacting polarizability:     1.773654866274112


As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.

### Manual solution of the Dyson equations
To see what goes on under the hood, we also show how to manually solve the
Dyson equation using KrylovKit:

In [8]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; scfres.ρ)
    χ0δV = apply_χ0(scfres, δV).δρ
    δρ - χ0δV
end

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(scfres, δVext).δρ

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.77e-03
[ Info: GMRES linsolve in iteration 1; step 3: normres = 2.85e-04
[ Info: GMRES linsolve in iteration 1; step 4: normres = 4.69e-06
[ Info: GMRES linsolve in iteration 1; step 5: normres = 1.09e-08
[ Info: GMRES linsolve in iteration 1; step 6: normres = 6.27e-11
[ Info: GMRES linsolve in iteration 1; step 7: normres = 6.44e-10
[ Info: GMRES linsolve in iteration 2; step 1: normres = 7.02e-11
┌ Info: GMRES linsolve converged at iteration 2, step 2:
│ * norm of residual = 4.01e-12
└ * number of operations = 11
Non-interacting polarizability: 1.9257125367671404
Interacting polarizability:     1.7736548619657102


We obtain the identical result to above.